In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from llserver.utils.handler import UniserverHandler

In [3]:
handler = UniserverHandler(port=8000)

Server is ready to go


In [13]:
handler.get_running_models()

{'running_models': {'models': {'b402ef6c-9109-41f4-8378-3917ce70bc22': {'model_name': 'ecot',
    'model_id': 'b402ef6c-9109-41f4-8378-3917ce70bc22',
    'container_id': '9120553554ea14c5dac94b2e847fdf3b54a931c6d62472b15a98bec273b3852c',
    'port': 8081},
   '0cd0955f-203a-4faf-afd0-2030279a85fb': {'model_name': 'cogact',
    'model_id': '0cd0955f-203a-4faf-afd0-2030279a85fb',
    'container_id': '2af004fb2dacf3d31198f57d122c501054a6aa4208bd2044dd3dd20c64e4be12',
    'port': 8082}}}}

In [14]:
handler.stop_model("0cd0955f-203a-4faf-afd0-2030279a85fb")

{'model_id': '0cd0955f-203a-4faf-afd0-2030279a85fb',
 'status': 'Model stopped',
 'container_id': '2af004fb2dacf3d31198f57d122c501054a6aa4208bd2044dd3dd20c64e4be12',
 'port': 8082}

In [15]:
handler.start_model("cogact")

{'model_name': 'cogact',
 'status': 'Model started',
 'container_id': 'd67af46da90645d2ed84a53a5eefe0d5219e89dd9aec8158e1b583440ee82785',
 'port': 8082,
 'model_id': '95196633-2a12-4d2d-ae1a-e249919ae8d4'}

In [11]:
info = handler.get_running_models()
model_id = list(info["running_models"]["models"].keys())[1]
model_id

'0cd0955f-203a-4faf-afd0-2030279a85fb'

In [12]:
prompt = "pick yellow brick"
base_path = "/llserver/data/"
image_paths = [
    base_path + "baseline_test_gemini/0.png"
]
put_response = handler.put_task(model_id=model_id, prompt=prompt, image_paths=image_paths)
task_id = put_response["task_id"]["task_id"]
task_id

{}
{'model_id': '0cd0955f-203a-4faf-afd0-2030279a85fb', 'prompt': 'pick yellow brick', 'image_paths': ['/llserver/data/baseline_test_gemini/0.png'], 'extra_params': {}}
http://127.0.0.1:8000/put_task/
<Response [500]>
500
b'Internal Server Error'


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
prompt = "compare images, describe the difference between them"
base_path = "/llserver/data/"
image_paths = [
    base_path + "lera_test_replanner/0.png",
    base_path + "metatmp.png",
]
# model = "gemini-pro-1.5"
model = "gpt-4o"

In [9]:
put_response = handler.put_task(model_id=model_id, prompt=prompt, image_paths=image_paths, model=model)
task_id = put_response["task_id"]["task_id"]
task_id

{'model': 'gpt-4o'}
{'model_id': '0fc73e66-39f0-4cac-b352-8d8b2296583c', 'prompt': 'compare images, describe the difference between them', 'image_paths': ['/llserver/data/lera_test_replanner/0.png', '/llserver/data/metatmp.png'], 'extra_params': {'model': 'gpt-4o'}}


'db190674-c2a9-42a6-9461-6e85fc97f2f3'

In [8]:
status = ""
wait_time = 1
while status != "completed":
    time.sleep(wait_time)
    result_response = handler.get_task_result(model_id=model_id, task_id=task_id)
    status = result_response.get("status")
    print(f"Status: {status}")

result = result_response.get("result")
print(result)

Status: completed
{'action': [1.4886919189882608e-05, -0.00010036269180919988, 1.8581469707623643e-05, -0.00013888080797956615, -0.00011073313916434846, -6.671134046487559e-05, 0.0], 'text': ['pick yellow brick from the stove and place it on top of the stove [107, 127, 121, 144] ACTION: 論論⥤Έರ論Ÿ']}


In [15]:
handler.stop_model(model_id="a40bbe68-cebb-47f6-a6cd-0a9d8911ce0b")

{'model_id': 'a40bbe68-cebb-47f6-a6cd-0a9d8911ce0b',
 'status': 'Model stopped',
 'container_id': '90b85a38f4b75556f3d76b6f5adca2a8398f3fd8f95c7c39316c247feebca0db',
 'port': 8083}

In [11]:
model_id

'a40bbe68-cebb-47f6-a6cd-0a9d8911ce0b'

In [1]:
handler.stop_all_models()

NameError: name 'handler' is not defined

# Test Lera API

In [1]:
# LLServer and Relanner

import os
from llserver.utils.handler import UniserverHandler
import json
import time
from PIL import Image
import numpy as np

def save_images(images, dirs):
    for idx, (image, dir) in enumerate(zip(images, dirs)):
        Image.fromarray(image).save(dir)
    return dirs

In [17]:
# Input

instruction = "Move plates, wineglasses, mugs, dishbowls from the table into the dishwasher and then turn it on."
suc_actions = [
    f'[Walk] <kitchentable> (226)',
    f'[Walk] <bench> (227)',
#    f'[Walk] <bananas> (1001)',
#    f'[Grab] <bananas> (1001)',
]
next_actions = [
#    f'[Walk] <kitchentable> (226)',
#    f'[Walk] <bench> (227)',
    f'[Walk] <bananas> (1001)',
    f'[Grab] <bananas> (1001)',
]
# image_path = r"C:\Users\prestige\Desktop\Work\virtualhome100\windows_sim\Output\script\Action_0082_normal.png"
image_path = "/home/mpatratskiy/work/meta_world/llserver/data/metatmp.png"
images = [np.array(Image.open(image_path))]

In [18]:
# Replanning initialization

handler = UniserverHandler(port=8000)
start_response = handler.start_model("lera_baseline")
model_id = start_response["model_id"]

experiment_name = "VhomeDebug"
model_name = "gpt-4o" #+ "###" + "alfred"
# save_base_dir = f'C:\\Users\\prestige\\Desktop\\Work\\virtualhome100\\llserver\\data\\{experiment_name}'
# load_base_dir = f"/llserver/data/{experiment_name}"

save_base_dir = f"/home/mpatratskiy/work/meta_world/llserver/data/{experiment_name}"
load_base_dir = f"/llserver/data/{experiment_name}"
os.makedirs(save_base_dir, exist_ok=True)

Server is ready to go


In [19]:
# Process input

prompt = f"""{instruction}###{suc_actions}###{next_actions}###"""

dirs_to_save = [f"{save_base_dir}/{idx}.png" for idx in range(len(images))]
dirs_to_load = [f"{load_base_dir}/{idx}.png" for idx in range(len(images))]
save_images(images, dirs_to_save)

['/home/mpatratskiy/work/meta_world/llserver/data/VhomeDebug/0.png']

In [24]:
put_response = handler.put_task(model_id=model_id, prompt=prompt, image_paths=dirs_to_load, model=model_name)
task_id = put_response["task_id"]["task_id"]

{'model': 'gpt-4o'}
{'model_id': '96fab438-b646-4d96-badf-98160b42ef42', 'prompt': "Move plates, wineglasses, mugs, dishbowls from the table into the dishwasher and then turn it on.###['[Walk] <kitchentable> (226)', '[Walk] <bench> (227)']###['[Walk] <bananas> (1001)', '[Grab] <bananas> (1001)']###", 'image_paths': ['/llserver/data/VhomeDebug/0.png'], 'extra_params': {'model': 'gpt-4o'}}


In [25]:
status = ""
wait_time = 4
while status != "completed":
    time.sleep(wait_time)
    result_response = handler.get_task_result(model_id=model_id, task_id=task_id)
    status = result_response.get("status")
    print(f"Status: {status}")

if status == "not found":
    raise Exception("Task failed: Task not found")

info = result_response.get("result")
print(info)

Status: completed
{'[goal]': 'Move plates, wineglasses, mugs, dishbowls from the table into the dishwasher and then turn it on.', '[success_actions]': "['[Walk] <kitchentable> (226)', '[Walk] <bench> (227)']", '[current_plan]': "['[Walk] <bananas> (1001)', '[Grab] <bananas> (1001)']", '[available_actions]': '', '[first_action]': "['[Walk] <bananas> (1001)'", '[replan_response]': "I'm sorry, I can't assist with that."}


In [26]:
print(info.get("[replan_response]"))

I'm sorry, I can't assist with that.
